# Omnifia Open-Domain Question-Answering Software

This software aims to answer users question like a chatbot based on their own data collected by Omnifia platform. Many services such as Gmail and slack are connected to the platform by which the software gets access to the data. 
The algorithm consists of three cosequtive parts:


1.   Refining data to label them in clusters
2.   Finding the most related context to the question
3.   Answering question based on the found context







In this part we install required libraries and mount google drive

In [ ]:
# Connect to Google Drive to reach files
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip install --quiet transformers
!pip install --quiet torch
!pip install --quiet tokenizers
!pip install --quiet sentencepiece
!pip install --quiet pytorch_lightning
!pip install --quiet datasets

     |████████████████████████████████| 4.0 MB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 582 kB 4.3 MB/s 
     |████████████████████████████████| 408 kB 42.3 MB/s 
     |████████████████████████████████| 136 kB 60.6 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
     |████████████████████████████████| 271 kB 55.4 MB/s 
     |████████████████████████████████| 144 kB 50.2 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 127 kB 42.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the pa

## 1. Clustering Data

In [ ]:
import os
import glob

In [ ]:
# Importing files
base_dir = os.path.join('./drive/MyDrive/text_data/text_data')
files = glob.glob(base_dir+'/*.clean')
len(files)

150

In [ ]:
# Import the questions
base_dir = os.path.join('./drive/MyDrive/text_data')
question_file = glob.glob(base_dir+'/*S10_question_answer_pairs.txt')
question_file

['./drive/MyDrive/text_data/S10_question_answer_pairs.txt']

In [ ]:
import numpy as np
import pandas as pd
import math
from random import random

In [ ]:
df_10 = pd.read_csv(question_file[0], sep='\t', encoding = 'ISO-8859-1')

In [ ]:
df_10

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,S10_set4_a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,S10_set4_a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,S10_set4_a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,S10_set4_a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,S10_set4_a10
...,...,...,...,...,...,...
1453,Zebra,What areas do the Grevy's Zebras inhabit?,NaN,hard,NaN,S10_set1_a9
1454,Zebra,Which species of zebra is known as the common ...,"Plains Zebra (Equus quagga, formerly Equus bur...",hard,medium,S10_set1_a9
1455,Zebra,Which species of zebra is known as the common ...,Plains Zebra,hard,medium,S10_set1_a9
1456,Zebra,At what age can a zebra breed?,five or six,hard,medium,S10_set1_a9


In [ ]:
df_10.drop_duplicates(subset='Question', keep='first', inplace=True)

In [ ]:
df_10

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,S10_set4_a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,S10_set4_a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,S10_set4_a10
6,Alessandro_Volta,Who did Alessandro Volta marry?,Alessandro Volta married Teresa Peregrini.,medium,medium,S10_set4_a10
8,Alessandro_Volta,What did Alessandro Volta invent in 1800?,"In 1800, Alessandro Volta invented the voltaic...",medium,easy,S10_set4_a10
...,...,...,...,...,...,...
1448,Zebra,What do zebras eat?,mainly grass,medium,hard,S10_set1_a9
1450,Zebra,What are zebras hunted for?,mainly for their skins,medium,medium,S10_set1_a9
1452,Zebra,What areas do the Grevy's Zebras inhabit?,semi-arid grasslands of Ethiopia and northern ...,hard,hard,S10_set1_a9
1454,Zebra,Which species of zebra is known as the common ...,"Plains Zebra (Equus quagga, formerly Equus bur...",hard,medium,S10_set1_a9


In [ ]:
files[:5]

['./drive/MyDrive/text_data/text_data/S09_set3_a5.txt.clean',
 './drive/MyDrive/text_data/text_data/S08_set3_a2.txt.clean',
 './drive/MyDrive/text_data/text_data/S10_set2_a10.txt.clean',
 './drive/MyDrive/text_data/text_data/S10_set6_a10.txt.clean',
 './drive/MyDrive/text_data/text_data/S08_set1_a5.txt.clean']

In [ ]:
df = pd.DataFrame(columns=['text', 'file_name'])

In [ ]:
df.head()

,text,file_name


In [ ]:
for f in files:
  single_file = open(f, encoding = "ISO-8859-1")
  text = single_file.read()
  df = df.append({'text': text, 'file_name': f}, ignore_index=True)

In [ ]:
df.head()

,text,file_name
0,Ottawa\n\n\n\nOttawa ( or sometimes ) is the...,./drive/MyDrive/text_data/text_data/S09_set3_a...
1,"James_Monroe\n\n\n\nJames Monroe (April 28, 17...",./drive/MyDrive/text_data/text_data/S08_set3_a...
2,Lyre\n\n\n\nThe lyre is a stringed musical in...,./drive/MyDrive/text_data/text_data/S10_set2_a...
3,"Gustav_Klimt\n\n \nGustav Klimt (July 14, 1862...",./drive/MyDrive/text_data/text_data/S10_set6_a...
4,elephant\n\n\n\nElephants (Elephantidae) are a...,./drive/MyDrive/text_data/text_data/S08_set1_a...


In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
texts = []

for text in df['text']:
  stemmed = ""
  token = word_tokenize(text)
  cleaned_token = []
  for word in token:
    if word not in stop_words:
        cleaned_token.append(word)

  for word in cleaned_token:
    texts.append(stemmer.stem(word))
    stemmed += stemmer.stem(word) + " "
  df['text'].replace(text, stemmed, inplace = True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
# initialize the vectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=0.00, max_df=0.95)
# fit_transform applies TF-IDF to clean texts - we save the array of vectors in X
X = vectorizer.fit_transform(df['text'])

In [ ]:
from sklearn.cluster import KMeans
# initialize kmeans with 5 score centroids
kmeans = KMeans(n_clusters=5, random_state=42)
# fit the model
kmeans.fit(X)
# store cluster labels in a variable
clusters = kmeans.labels_

In [ ]:
from sklearn.decomposition import PCA

# initialize PCA with 2 components
pca = PCA(n_components=2, random_state=42)
# pass our X to the pca and store the reduced vectors into pca_vecs
pca_vecs = pca.fit_transform(X.toarray())
# save our two dimensions into x0 and x1
x0 = pca_vecs[:, 0]
x1 = pca_vecs[:, 1]

In [ ]:
# assign clusters and pca vectors to our dataframe 
df['clusters'] = clusters
df['x0'] = x0
df['x1'] = x1

In [ ]:
df.head()

,text,file_name,clusters,x0,x1
0,ottawa ottawa ( sometim ) capit canada countri...,./drive/MyDrive/text_data/text_data/S09_set3_a...,0,-0.197163,-0.051123
1,"james_monro jame monro ( april 28 , 1758 â j...",./drive/MyDrive/text_data/text_data/S08_set3_a...,4,-0.024696,-0.173897
2,lyre lyre string music instrument well known u...,./drive/MyDrive/text_data/text_data/S10_set2_a...,1,0.198192,0.257258
3,"gustav_klimt gustav klimt ( juli 14 , 1862 â...",./drive/MyDrive/text_data/text_data/S10_set6_a...,4,0.019887,-0.078494
4,eleph eleph ( elephantida ) famili order probo...,./drive/MyDrive/text_data/text_data/S08_set1_a...,3,-0.093380,0.092693


In [ ]:
df['clusters'].value_counts()

0    50
4    35
3    30
1    20
2    15
Name: clusters, dtype: int64

In [ ]:
def get_top_keywords(n_terms):
    """This function returns the keywords for each centroid of the KMeans"""
    df = pd.DataFrame(X.todense()).groupby(clusters).mean() # groups the TF-IDF vector by cluster
    terms = vectorizer.get_feature_names_out() # access tf-idf terms
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(r)[-n_terms:]])) # for each row of the dataframe, find the n terms that have the highest tf idf score
            
get_top_keywords(10)


Cluster 0
countri,univers,offici,region,popul,citi,spoken,dialect,languag,the

Cluster 1
play,tone,sound,vibrat,player,pitch,music,string,tune,instrument

Cluster 2
radioact,phosphoresc,avogadro,volta,paper,uppsala,experi,he,becquerel,celsiu

Cluster 3
eat,mammal,hunt,predat,prey,anim,male,the,femal,speci

Cluster 4
john,friend,jame,portrait,life,father,the,work,hi,he


In [ ]:
df_10 = df_10.reset_index()

### Trying To Find Overall Accuracy

In [ ]:
number_of_correct_clusters = 0
for index, ques in enumerate(df_10['Question'][:50]):
  token = word_tokenize(ques)
  cleaned_token = []
  for word in token:
    if word not in stop_words:
        cleaned_token.append(word)
  stemmed = ""
  for word in token:
    stemmed += stemmer.stem(word) + " "

  question = [stemmed]
  X_new_tfidf = vectorizer.transform(question)
  y = kmeans.predict(X_new_tfidf)
  y[0]

  filename=[]
  for file_name in df['file_name'][df['clusters'] == y[0]]:
    filename.append(file_name)

  punctuations = '''!()-[]{};:'"\,\n<>./?@#$%^&*_~'''
  count = 1;
  Documents_dict = {}
  vocab_dict = {}
  for text in df['text'][df['clusters'] == y[0]]:
      final_text = ""
      for char in text:
        if char not in punctuations:
          final_text = final_text + char
      list_text = final_text.split(" ")
      for word in list_text:
        if word in vocab_dict.keys():
          vocab_dict[word] = vocab_dict[word] + 1
        else:
          vocab_dict[word] = 1
      Length_text = len(list_text)
      uniq_set = set(list_text)
      uniq_text = (list(uniq_set))
      #print("Document - ", df['file_name'][count-1], ":")
      #print(final_text, "\n")
      TF_dict = {}
      #print("TERMS", "->", "TERM FREQ", "->", "NORM TERM FREQ  (term freq/ doc size) ", "\n")
      norm_doc = 0;
      for words in uniq_text:
        list_temp = [list_text.count(words), list_text.count(words)/Length_text]
        TF_dict[words] = list_temp
        norm_doc = norm_doc + TF_dict[words][1]*TF_dict[words][1]
      norm_docc = math.sqrt(norm_doc)
      TF_dict["norm_doc"+str(count)] = norm_docc
      #print("\n|| d"+str(count)+" || = ", norm_docc)
      Documents_dict["doc"+str(count)] = TF_dict
      #print("\n")
      count = count + 1


  freq_vocab = []
  uniq_vocab = []
  for word in vocab_dict.keys():
    if vocab_dict[word] > 10:
      freq_vocab.append(word)
    elif vocab_dict[word] == 1:
      uniq_vocab.append(word)
  #print("High frequency words :")
  #print(freq_vocab, "\n")
  #print("Rare words:")
  #print(uniq_vocab, "\n")
  count = count - 1
  query = stemmed
  query = query.split(" ")
  query_set = set(query)
  query_list = (list(query_set))
  i=0
  Query_doc_freq = {}
  Total_doc_freq = 0
  while i<len(query_list):
    doc_freq = 0
    j=1
    while j<=count:
      if query_list[i] in Documents_dict["doc"+str(j)].keys():
        doc_freq = doc_freq + 1
      j=j+1
    Total_doc_freq = Total_doc_freq + doc_freq
    Query_doc_freq[query_list[i]] = doc_freq
    i = i+1
  if Total_doc_freq == 0:
    print("")
    #print("--------Query not found--------")
  else:
    #print("\n\nTERMS", "->", "DOC FREQ", "->", "NORM DOC FREQ  (doc freq/total doc freq)\n")
    norm_q = 0
    for Keyss in Query_doc_freq:
      #print(Keyss, "->", Query_doc_freq[Keyss], "->", Query_doc_freq[Keyss]/Total_doc_freq)
      Query_doc_freq[Keyss] = Query_doc_freq[Keyss]/Total_doc_freq
      norm_q = norm_q + Query_doc_freq[Keyss]*Query_doc_freq[Keyss]
    norm_qq = math.sqrt(norm_q)
    #print("\n|| q || = ", norm_qq, "\n")
    Similarity_scores = {}
    i=1
    while i<= count:
      sim_temp = 0
      j=0
      while j<len(query_list):
        if query_list[j] in Documents_dict["doc"+str(i)].keys():
          sim_temp = sim_temp + Documents_dict["doc"+str(i)][query_list[j]][1]*Query_doc_freq[query_list[j]]
        j=j+1
      denominator = norm_qq*Documents_dict["doc"+str(i)]["norm_doc"+str(i)]
      Similarity_scores[filename[i-1]] = sim_temp/denominator
      i=i+1
    # print(vocab_dict)
    #print("Similarity scores:" )
    #print(Similarity_scores)
    #print("\nRanking based on similarity scores: ")
    #print (sorted(Similarity_scores.items(), reverse=True, key = lambda x : x[1])[:10])
    #print("\n")

  random_question_file_name = df_10['ArticleFile'][index]
  random_question_file_name

  for a in sorted(Similarity_scores.items(), reverse=True, key = lambda x : x[1])[:10]:
    sim_file_name, sim_score = a
    if random_question_file_name in sim_file_name:
      number_of_correct_clusters += 1

print((number_of_correct_clusters / 50))

0.7


# 2. Ranking contexts for a question

### Trying For One Question

In [ ]:
value = int(random() * len(df_10))
random_question = df_10['Question'][value]
token = word_tokenize(random_question)
cleaned_token = []
for word in token:
  if word not in stop_words:
      cleaned_token.append(word)
stemmed = ""
for word in token:
  stemmed += stemmer.stem(word) + " "

In [ ]:
random_question

'Is the most important river the Ciliwung River'

In [ ]:
random_question_answer = df_10['Answer'][value]
random_question_answer

'yes'

In [ ]:
question = [stemmed]
X_new_tfidf = vectorizer.transform(question)
y = kmeans.predict(X_new_tfidf)
y[0]

4

In [ ]:
filename=[]
for file_name in df['file_name'][df['clusters'] == y[0]]:
  filename.append(file_name)

In [ ]:
punctuations = '''!()-[]{};:'"\,\n<>./?@#$%^&*_~'''
count = 1;
Documents_dict = {}
vocab_dict = {}
for text in df['text'][df['clusters'] == y[0]]:
    final_text = ""
    for char in text:
    	if char not in punctuations:
    		final_text = final_text + char
    list_text = final_text.split(" ")
    for word in list_text:
    	if word in vocab_dict.keys():
    		vocab_dict[word] = vocab_dict[word] + 1
    	else:
    		vocab_dict[word] = 1
    Length_text = len(list_text)
    uniq_set = set(list_text)
    uniq_text = (list(uniq_set))
    print("Document - ", df['file_name'][count-1], ":")
    print(final_text, "\n")
    TF_dict = {}
    print("TERMS", "->", "TERM FREQ", "->", "NORM TERM FREQ  (term freq/ doc size) ", "\n")
    norm_doc = 0;
    for words in uniq_text:
    	list_temp = [list_text.count(words), list_text.count(words)/Length_text]
    	TF_dict[words] = list_temp
    	norm_doc = norm_doc + TF_dict[words][1]*TF_dict[words][1]
    norm_docc = math.sqrt(norm_doc)
    TF_dict["norm_doc"+str(count)] = norm_docc
    print("\n|| d"+str(count)+" || = ", norm_docc)
    Documents_dict["doc"+str(count)] = TF_dict
    print("\n")
    count = count + 1


freq_vocab = []
uniq_vocab = []
for word in vocab_dict.keys():
	if vocab_dict[word] > 10:
		freq_vocab.append(word)
	elif vocab_dict[word] == 1:
		uniq_vocab.append(word)
print("High frequency words :")
print(freq_vocab, "\n")
print("Rare words:")
print(uniq_vocab, "\n")
count = count - 1
query = stemmed
query = query.split(" ")
query_set = set(query)
query_list = (list(query_set))
i=0
Query_doc_freq = {}
Total_doc_freq = 0
while i<len(query_list):
	doc_freq = 0
	j=1
	while j<=count:
		if query_list[i] in Documents_dict["doc"+str(j)].keys():
			doc_freq = doc_freq + 1
		j=j+1
	Total_doc_freq = Total_doc_freq + doc_freq
	Query_doc_freq[query_list[i]] = doc_freq
	i = i+1
if Total_doc_freq == 0:
	print("--------Query not found--------")
else:
	print("\n\nTERMS", "->", "DOC FREQ", "->", "NORM DOC FREQ  (doc freq/total doc freq)\n")
	norm_q = 0
	for Keyss in Query_doc_freq:
		print(Keyss, "->", Query_doc_freq[Keyss], "->", Query_doc_freq[Keyss]/Total_doc_freq)
		Query_doc_freq[Keyss] = Query_doc_freq[Keyss]/Total_doc_freq
		norm_q = norm_q + Query_doc_freq[Keyss]*Query_doc_freq[Keyss]
	norm_qq = math.sqrt(norm_q)
	print("\n|| q || = ", norm_qq, "\n")
	Similarity_scores = {}
	i=1
	while i<= count:
		sim_temp = 0
		j=0
		while j<len(query_list):
			if query_list[j] in Documents_dict["doc"+str(i)].keys():
				sim_temp = sim_temp + Documents_dict["doc"+str(i)][query_list[j]][1]*Query_doc_freq[query_list[j]]
			j=j+1
		denominator = norm_qq*Documents_dict["doc"+str(i)]["norm_doc"+str(i)]
		Similarity_scores[filename[i-1]] = sim_temp/denominator
		i=i+1
	# print(vocab_dict)
	print("Similarity scores:" )
	print(Similarity_scores)
	print("\nRanking based on similarity scores: ")
	print (sorted(Similarity_scores.items(), reverse=True, key = lambda x : x[1])[:10])
	print("\n")

Document -  ./drive/MyDrive/text_data/text_data/S09_set1_a2.txt.clean :
lion the lion  panthera leo  member famili felida one four big cat genu panthera  with except larg male exceed 250 kg  550 lb  weight  secondlargest live cat tiger  wild lion current exist subsaharan africa asia critic endang remnant popul northwest india  disappear north africa  middl east  western asia histor time  until late pleistocen  10000 year ago   lion widespread larg land mammal besid human  they found africa  much eurasia western europ india  america  yukon peru  should surviv rigor cubhood  lioness secur habitat kruger nation park may frequent reach age 12â14 year wherea lion seldom live longer 8 year  howev  record lioness live 20 year wild  In captiv male femal lion live 20 year  they typic inhabit savanna grassland  although may take bush forest  lion unusu social compar cat  A pride lion consist relat femal offspr small number adult male  group femal lion typic hunt togeth  prey mostli larg ungul 

In [ ]:
sorted(Similarity_scores.items(), reverse=True, key = lambda x : x[1])[:10]

[('./drive/MyDrive/text_data/text_data/S08_set2_a5.txt.clean',
  0.6519944898766685),
 ('./drive/MyDrive/text_data/text_data/S10_set1_a3.txt.clean',
  0.6474633470861494),
 ('./drive/MyDrive/text_data/text_data/S10_set6_a5.txt.clean',
  0.6377099446696298),
 ('./drive/MyDrive/text_data/text_data/S10_set1_a10.txt.clean',
  0.637582585079464),
 ('./drive/MyDrive/text_data/text_data/S08_set1_a1.txt.clean',
  0.6364240684959617),
 ('./drive/MyDrive/text_data/text_data/S10_set6_a4.txt.clean',
  0.6364179341444031),
 ('./drive/MyDrive/text_data/text_data/S10_set6_a10.txt.clean',
  0.6361165589316266),
 ('./drive/MyDrive/text_data/text_data/S10_set4_a2.txt.clean',
  0.6359769390884522),
 ('./drive/MyDrive/text_data/text_data/S10_set6_a6.txt.clean',
  0.63467425300023),
 ('./drive/MyDrive/text_data/text_data/S10_set6_a2.txt.clean',
  0.6335435672124567)]

In [ ]:
random_question_file_name = df_10['ArticleFile'][value]
random_question_file_name

'S10_set3_a5'

In [ ]:
for a in sorted(Similarity_scores.items(), reverse=True, key = lambda x : x[1])[:10]:
  sim_file_name, sim_score = a
  if random_question_file_name in sim_file_name:
    print("We Found The Document")

In [ ]:
# Concating the files to one context
from pathlib import Path

related_texts = ''
for a in sorted(Similarity_scores.items(), reverse=True, key = lambda x : x[1])[:10]:
  txt = Path(a[0]).read_text()
  related_texts += txt
  related_texts += '\n'
print('question:', random_question)
print('context:', related_texts[:150])

question: Is the most important river the Ciliwung River
context: Ghana



The Republic of Ghana is a country in West Africa. It borders Côte d'Ivoire (also known as Ivory Coast) to the west, Burkina Faso to the nort


In [ ]:
random_question_contex = related_texts

# 3. Predicting the Answer

In this section we try to create our Transfer Learning model to predict the answer to the chosen question based on the contexts found in the previous section. The model we use is T5 which is introduced by Google in 2020. The model is trained with C4 dataset, but in this software we try to train it with SQUAD dataset which is specifically used for question-answering purposes.
Also, it would be better if we had some answered questions with our own data, but this job seems quite immpossible as Omnifia users data are not supposed to be analysed.

Due to the limitaion of the hardware, we use T5 base model which has a fewer parameters. Also, we use Pytorch and Pytorch lightnings classes as it creates a more comprehensive and readable class definitions for datasets, modules and models.

## Global Vars

In [ ]:
squad_v2 = False
MODEL_NAME ='t5-base'
BATCH_SIZE = 4
N_EPOCHS = 2
NUM_WORKERS = 2

## Importing Dataset (SQUAD)

Squad v2 dataset includes some unanswerable question. To handle them, we need to have a stronger hardware. So we skill that for now and use Squad v1.1 instead. 

In [ ]:
from datasets import load_dataset
dataset = load_dataset("squad_v2" if squad_v2 else "squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

## Imports

In [ ]:

import glob
import os
import json
import time
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split

from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)

Global seed set to 0


0

## Train Validation Split

We also use a train-test split to prevent overfitting and have better training.

In [ ]:
def clean_data(data_df):
  data_df.dropna(inplace=True)
  data_rows = []
  for i in range(0, len(data_df)):
    data = data_df.iloc[i]
    question = data['question']
    context = data['context']
    answers = data['answers']
    answer_text = data['answers']['text'][0]
    answer_start = data['answers']['answer_start'][0]
    answer_end = answer_start + len(answer_text)
          
    data_rows.append({
        "question" : question,
        "context"  : context,
        "answer_text" : answer_text,
        "answer_start" : answer_start,
        "answer_end" : answer_end
    })
  return pd.DataFrame(data_rows)

In [ ]:
data_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['validation'])

In [ ]:
data_df.iloc[1000]['answers']

{'answer_start': [190], 'text': ['$250,000']}

In [ ]:
data_df = clean_data(data_df)
test_df = clean_data(test_df)

In [ ]:
data_df.head()

,question,context,answer_text,answer_start,answer_end
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,515,541
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,188,213
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",the Main Building,279,296
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,381,420
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,92,126


In [ ]:
train_df, val_df = train_test_split(data_df, test_size=0.05)

In [ ]:
train_df.shape,  val_df.shape

((83219, 5), (4380, 5))

## Pytorch Lightning Dataset, DataModule, LightningDataModule 

T5 model needs a tokenized input ids and target ids which are fetched from the question + context and the answers respectively. So, we use T5 tokenizer that is quite fast.

#### Pytorch lightning Dataset
This class is defined to handle everything related to the dataset which is mainly tokenizing them. 

In [ ]:
class SquadDataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,

      ):
    
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len


  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    labels = target_encoding['input_ids']
    labels[labels==0] = -100

    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(), # Flatten for batching
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )

#### Lightining Data Module
To load train and test datasets

In [ ]:
class SquadDataModule(pl.LightningDataModule):
  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer:T5Tokenizer,
      batch_size: int = 8,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = SquadDataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
        )

    self.test_dataset = SquadDataset(
    self.test_df,
    self.tokenizer,
    self.source_max_token_len,
    self.target_max_token_len
    )
 
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=NUM_WORKERS
        )
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=NUM_WORKERS
        )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=1,
        num_workers=NUM_WORKERS
        )

#### Lightning Model
For training and validation steps

In [ ]:
class SquadModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(
        MODEL_NAME,
         return_dict=True)

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss)
    return {"loss": loss, "predictions":outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss)
    return loss

  # Adamw optimzer is mainly used in question answering
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer

## Tokenization

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

In [ ]:
data_module = SquadDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

### Model Creation and Fine-Tuning

In [ ]:
model = SquadModel()

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
# To record the best performing model using checkpoint

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

#### Pytorch Lighning Trainer

In [ ]:
trainer = pl.Trainer(
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate = 30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


#### Fitting the model with trainer

In [ ]:
trainer.fit(model, data_module)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 20805: 'val_loss' reached 0.27440 (best 0.27440), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 1, global step 41610: 'val_loss' was not in top 1


In [ ]:
trained_model = SquadModel.load_from_checkpoint("./drive/MyDrive/best-checkpoint.ckpt")
trained_model.freeze()

In [ ]:
test_module = SquadDataModule(data_df, test_df, tokenizer, batch_size=BATCH_SIZE)
test_module.setup()

In [ ]:
test_result = trainer.test(model, test_module)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            8.560860633850098
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
test_result

[{'test_loss': 8.560860633850098}]

### Prediction

To output a prediction we need to generate the prediction which is a tokenized vector. So we need a decoder to convert it to words.

In [ ]:
def predict(question):
  source_encoding=tokenizer(
      question["question"],
      question['context'],
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )

  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]

  return "".join(preds)

In [ ]:
from sklearn.metrics import f1_score   
preds = []
for r in range(0, len(test_df)):
  row = test_df.iloc[r]
  pred = predict(row)
  preds.append(pred)
preds
#f1_score = f1_score(test_df['answer_text'], preds)
#f1_score

['Denver Broncos',
 'Carolina Panthers',
 "Levi's Stadium",
 'Carolina Panthers',
 'gold',
 'golden anniversary',
 'February 7, 2016',
 'American Football Conference',
 'golden anniversary',
 'American Football Conference',
 'February 7, 2016',
 'Carolina Panthers',
 "Levi's Stadium",
 'Santa Clara, California',
 'Super Bowl L',
 '2015',
 '2016',
 'Santa Clara, California',
 "Levi's Stadium",
 '24–10',
 'February 7, 2016',
 '2015',
 'Denver Broncos',
 'Carolina Panthers',
 'Carolina Panthers',
 '2015',
 'Denver Broncos',
 "Levi's Stadium",
 'Super Bowl 50',
 'Denver Broncos',
 'Cam Newton',
 'eight',
 '1995',
 'Arizona Cardinals',
 'New England Patriots',
 'Arizona Cardinals',
 'New England Patriots',
 'New England Patriots',
 'four',
 'Cam Newton',
 '15–1',
 'Cam Newton',
 '12–4',
 'four',
 'New England Patriots',
 'Cam Newton',
 'Arizona Cardinals',
 'second',
 'Arizona Cardinals',
 'Cam Newton',
 'Arizona Cardinals',
 'Arizona Cardinals',
 'Cam Newton',
 'Arizona Cardinals',
 '1995'

In [ ]:
f1_score = f1_score(test_df['answer_text'], preds, average='weighted')
f1_score

0.5649749658103382

Now let's take a look at the predictoin of our own question.

In [ ]:
our_question = {
    'question': random_question,
    'context': random_question_contex
}
our_question

{'context': 'Ghana\n\n\n\nThe Republic of Ghana is a country in West Africa. It borders Côte d\'Ivoire (also known as Ivory Coast) to the west, Burkina Faso to the north, Togo to the east, and the Gulf of Guinea to the south. The word "Ghana" means "Warrior King", Jackson, John G. Introduction to African Civilizations, 2001. Page 201.  and was the source of the name "Guinea" (via French Guinoye) used to refer to the West African coast (as in Gulf of Guinea).\n\nGhana was inhabited in pre-colonial times by a number of ancient kingdoms, including the Ga Adangbes on the eastern coast, inland Empire of Ashanti and various Fante states along the coast and inland. Trade with European states flourished after contact with the Portuguese in the 15th century, and the British established a crown colony, Gold Coast, in 1874. MacLean, Iain. Rational Choice and British Politics: An Analysis of Rhetoric and Manipulation from Peel to Blair, 2001. Page 76.\n\nUpon achieving independence from the United

In [ ]:
prediction = predict(our_question)
prediction

'the Ciliwung River'

In [ ]:
random_question_answer

'yes'